In [226]:
import requests
import json
import pandas as pd

In [208]:
#Here are a few coordinates, and you have to find the fortnightly average temperature for these coordinates(longitude and latitude):
lat_long_deg_min_sec = [
('37° 52\' 5.336" S', '144° 55\' 40.452" E'),
('38° 19\' 57.125" S', '144° 54\' 17.831" E'),
('38° 51\' 34.878" S', '142° 54\' 18.281" E'), 
('37° 50\' 33.481" S', '135° 38\' 20.388" E'),
('35° 5\' 50.501" S', '128° 48\' 40.243" E'), 
('34° 20\' 7.398" S', '121° 35\' 19.757" E'),
('34° 55\' 34.99" S', '118° 45\' 4.596" E'), 
('35° 3\' 36.441" S', '115° 51\' 54.355" E'),
('30° 45\' 16.173" S', '110° 26\' 25.532" E'),
('12° 5\' 45.683" S', '106° 25\' 8.984" E'),
('6° 18\' 54.271" S', '105° 33\' 56.07" E'), 
('5° 15\' 14.261" S', '106° 13\' 37.411" E'),
('3° 42\' 37.526" S', '107° 35\' 13.608" E'), 
('2° 0\' 50.122" S', '107° 12\' 50.324" E'),
('0° 42\' 51.809" N', '105° 19\' 28.118" E'), 
('1° 18\' 56.371" N', '104° 19\' 31.582" E'),
('1° 13\' 14.328" N', '103° 52\' 6.734" E')
]

In [209]:
import re
def convert(tude):
    multiplier = 1 if tude[-1] in ['N', 'E'] else -1
    tude = re.split('° |\' |" ', tude)
    #print(tude)
    return multiplier * sum(float(x) / 60 ** n for n, x in enumerate(tude[:-1]))

In [210]:
lat_long_dec = []

for lat_long in lat_long_deg_min_sec:
    lat = convert(lat_long[0])
    long = convert(lat_long[1])
    lat_long_dec.append((lat, long))

print(lat_long_dec)

[(-37.86814888888889, 144.92790333333332), (-38.33253472222223, 144.90495305555555), (-38.85968833333334, 142.90507805555555), (-37.84263361111111, 135.63899666666666), (-35.09736138888889, 128.81117861111113), (-34.335388333333334, 121.58882138888889), (-34.92638611111111, 118.75127666666667), (-35.0601225, 115.86509861111111), (-30.7544925, 110.44042555555556), (-12.096023055555555, 106.41916222222223), (-6.315075277777778, 105.565575), (-5.2539613888888885, 106.22705861111112), (-3.710423888888889, 107.58711333333333), (-2.0139227777777777, 107.21397888888889), (0.7143913888888889, 105.32447722222221), (1.3156586111111113, 104.32543944444444), (1.2206466666666669, 103.86853722222222)]


In [211]:
CLIENT_ID = '50579375f5d1481e85e161222211406'

for lat, long in lat_long_dec:
    print(lat, long, '-----------------------------------------------------------')

    # create the API request URL
    url = 'https://api.worldweatheronline.com/premium/v1/weather.ashx?key={}&q={},{}&format=json&num_of_days={}'.format(
        CLIENT_ID,
        lat, 
        long, 
        5)
    
    response  = requests.get(url)
    forecasts  = response.json()
    
    try:
        print("Date: {}, Max Temperature: {}".format(forecasts['data']['weather'][0]['date'], forecasts['data']['weather'][0]['maxtempC']))
    except:
        print('Empty Response')
    

-37.86814888888889 144.92790333333332 -----------------------------------------------------------
Date: 2021-06-24, Max Temperature: 15
-38.33253472222223 144.90495305555555 -----------------------------------------------------------
Date: 2021-06-24, Max Temperature: 15
-38.85968833333334 142.90507805555555 -----------------------------------------------------------
Date: 2021-06-24, Max Temperature: 12
-37.84263361111111 135.63899666666666 -----------------------------------------------------------
Date: 2021-06-24, Max Temperature: 13
-35.09736138888889 128.81117861111113 -----------------------------------------------------------
Empty Response
-34.335388333333334 121.58882138888889 -----------------------------------------------------------
Date: 2021-06-24, Max Temperature: 14
-34.92638611111111 118.75127666666667 -----------------------------------------------------------
Date: 2021-06-24, Max Temperature: 12
-35.0601225 115.86509861111111 ---------------------------------------

In [212]:
!pip install wwo-hist

In [213]:
from wwo_hist import retrieve_hist_data

In [214]:
frequency=24
start_date = '1-JUN-2016'
end_date = '1-AUG-2016'
api_key = CLIENT_ID



hist_weather_data_df = pd.DataFrame()

for lat, long in lat_long_dec:
    location_list = []
    
    location_list.append('{},{}'.format(lat, long))
    
    try:
        hist_weather_data_list = retrieve_hist_data(api_key,
                                    location_list,
                                    start_date,
                                    end_date,
                                    frequency,
                                    location_label = False,
                                    export_csv = False,
                                    store_df = True)
        
        # hist_weather_data_list is a list of datframes. Since we are making one call for one location, we get the first dataframe as the first element of the list
        hist_weather_data_df = hist_weather_data_df.append(hist_weather_data_list[0])
    
    except:
        print('Unable to find any matching weather for location {}'.format(location_list))



Retrieving weather data for -37.86814888888889,144.92790333333332


Currently retrieving data for -37.86814888888889,144.92790333333332: from 2016-06-01 to 2016-06-30
Time elapsed (hh:mm:ss.ms) 0:00:01.408148
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2016-07-01 to 2016-07-31
Time elapsed (hh:mm:ss.ms) 0:00:02.844624
Currently retrieving data for -37.86814888888889,144.92790333333332: from 2016-08-01 to 2016-08-01
Time elapsed (hh:mm:ss.ms) 0:00:03.157949


Retrieving weather data for -38.33253472222223,144.90495305555555


Currently retrieving data for -38.33253472222223,144.90495305555555: from 2016-06-01 to 2016-06-30
Time elapsed (hh:mm:ss.ms) 0:00:01.382130
Currently retrieving data for -38.33253472222223,144.90495305555555: from 2016-07-01 to 2016-07-31
Time elapsed (hh:mm:ss.ms) 0:00:03.213461
Currently retrieving data for -38.33253472222223,144.90495305555555: from 2016-08-01 to 2016-08-01
Time elapsed (hh:mm:ss.ms) 0:00:03.504483


Retrieving w

In [237]:
hist_weather_data_df['date_time'] = pd.to_datetime(hist_weather_data_df['date_time'])
hist_weather_data_df['tempC'] = hist_weather_data_df['tempC'].astype('int')

In [216]:
hist_weather_data_df.reset_index(inplace=True, drop=True)
hist_weather_data_df

,date_time,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,moonrise,moonset,sunrise,...,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph,location
0,2016-06-01,14,9,0.0,7.0,3,12,02:25 AM,02:30 PM,07:27 AM,...,13,64,80,0.4,1025,14,9,178,8,"-37.86814888888889,144.92790333333332"
1,2016-06-02,16,10,0.0,8.7,4,4,03:34 AM,03:10 PM,07:28 AM,...,8,6,75,0.0,1027,16,10,124,5,"-37.86814888888889,144.92790333333332"
2,2016-06-03,15,9,0.0,8.7,3,0,04:44 AM,03:53 PM,07:28 AM,...,13,33,78,1.1,1024,15,9,105,9,"-37.86814888888889,144.92790333333332"
3,2016-06-04,15,12,0.0,3.7,3,0,05:55 AM,04:41 PM,07:29 AM,...,12,90,92,12.3,1015,15,6,135,8,"-37.86814888888889,144.92790333333332"
4,2016-06-05,15,12,0.0,3.7,3,0,07:03 AM,05:33 PM,07:30 AM,...,12,89,90,0.9,1005,15,8,236,8,"-37.86814888888889,144.92790333333332"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,2016-07-28,32,27,0.0,11.6,7,34,12:38 AM,01:06 PM,06:06 AM,...,13,33,69,1.8,1009,32,10,169,9,"1.2206466666666669,103.86853722222222"
864,2016-07-29,32,26,0.0,11.6,7,26,01:33 AM,02:02 PM,06:06 AM,...,12,23,70,1.5,1009,32,10,177,9,"1.2206466666666669,103.86853722222222"
865,2016-07-30,33,26,0.0,11.6,7,19,02:29 AM,02:59 PM,06:06 AM,...,9,27,70,2.2,1009,33,10,158,7,"1.2206466666666669,103.86853722222222"
866,2016-07-31,33,26,0.0,11.6,7,15,03:26 AM,03:57 PM,06:06 AM,...,8,29,72,7.2,1009,33,10,160,6,"1.2206466666666669,103.86853722222222"


In [217]:
grouped = hist_weather_data_df.groupby('location')
grouped

In [218]:
!pip install folium

In [219]:
!pip install vincent

In [220]:
import folium
import vincent

# To initialize vincent in the notebook
vincent.core.initialize_notebook()

In [258]:
center_lat = sum(value[0] for value in lat_long_dec) / len(lat_long_dec)
center_long = sum(value[1] for value in lat_long_dec) / len(lat_long_dec)

m = folium.Map(location=[center_lat, center_long], zoom_start=4)

for name, group in grouped:
    print(name)
    
    loc_df = group[['date_time', 'tempC']].set_index('date_time')
    
    five_years_line = vincent.Line(loc_df, width=300, height=150)
    five_years_line.axis_titles(x='Date', y='tempC')
    #five_years_line.display()
    
    five_years_vega = folium.Vega(five_years_line.to_json(), width=350, height=200)
    
    lat = name.split(',')[0]
    long = name.split(',')[1]
    
    marker = folium.Marker(
            location = [lat, long],
            fill_opacity = 0.8, color=None,
            fill_color = ('#274cc9'),
            popup = folium.Popup().add_child(five_years_vega),
            tooltip = '{}, {}'.format(lat, long)
            )

    marker.add_to(m)

m

-2.0139227777777777,107.21397888888889
-3.710423888888889,107.58711333333333
-34.335388333333334,121.58882138888889
-34.92638611111111,118.75127666666667
-35.0601225,115.86509861111111
-37.84263361111111,135.63899666666666
-37.86814888888889,144.92790333333332
-38.33253472222223,144.90495305555555
-38.85968833333334,142.90507805555555
-5.2539613888888885,106.22705861111112
-6.315075277777778,105.565575
0.7143913888888889,105.32447722222221
1.2206466666666669,103.86853722222222
1.3156586111111113,104.32543944444444


'folium' is not recognized as an internal or external command,
operable program or batch file.
